In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = None


In [2]:
# Parameters
product = {"nb": "/home/jerem/intention1.0/output/Carnet_de_Curiosite_ml.ipynb"}


In [3]:
import pdfplumber
import PyPDF2
from PIL import Image
import re
import io
import base64
import nbformat


In [4]:
import os
import nbformat
from nbformat.v4 import new_code_cell, new_markdown_cell
import pdfplumber
import re
import io
import base64

# Load or initialize notebook
if os.path.exists("/home/jerem/Desktop/Carnet_de_Curiosite_ml.ipynb"):
    nb = nbformat.read("/home/jerem/Desktop/Carnet_de_Curiosite_ml.ipynb", as_version=4)
else:
    nb = nbformat.v4.new_notebook()

new_cells = []

# Process PDF pages
with pdfplumber.open("/home/jerem/Desktop/Carnet de Curiosité _ Intention, Décision & Curiosité – Un voyage interdisciplinaire.pdf") as pdf:
    for i, page in enumerate(pdf.pages, start=1):
        # Extract words for proper spacing
        words = page.extract_words()
        if words:
            words_sorted = sorted(words, key=lambda x: (x['top'], x['x0']))
            text = ' '.join(w['text'] for w in words_sorted)
        else:
            text = page.extract_text() or ""
        # Page header
        new_cells.append(new_markdown_cell(f"## Page {i}\n"))
        # Detect and split by section titles
        pattern = re.compile(r"(^[A-Z0-9].*)", re.M)
        parts = pattern.split(text)
        for idx in range(1, len(parts), 2):
            title = parts[idx].strip()
            body = parts[idx+1].strip()
            md = f"# {title}\n\n{body}"
            new_cells.append(new_markdown_cell(md))
        # Extract images
        for img in page.images:
            im = page.within_bbox((img["x0"], img["top"], img["x1"], img["bottom"])).to_image(resolution=300).original
            buf = io.BytesIO()
            im.save(buf, format="PNG")
            data = base64.b64encode(buf.getvalue()).decode("utf-8")
            img_md = f"![](data:image/png;base64,{data})"
            new_cells.append(new_markdown_cell(img_md))

# Insert ML scaffolding after first cell
new_cells.insert(1, new_code_cell("# Data Preprocessing Placeholder\n# TODO: Add data preprocessing steps"))

# Append cells and set metadata
nb.cells.extend(new_cells)
nb.metadata.kernelspec = {
    "display_name": "Python 3",
    "language": "python",
    "name": "python3"
}
nb.metadata.language_info = {
    "name": "python",
    "version": "3"
}

# Write notebook
nbformat.write(nb, "/home/jerem/Desktop/Carnet_de_Curiosite_ml.ipynb")
